Dimensionality reduction in various ways to lower the number of nodes into one that works for Gaussian processes
- UMAPs / TSNE
- GP latent variable model https://pyro.ai/examples/gplvm.html

In [2]:
import pandas as pd
import numpy as np
import umap
from sklearn.manifold import TSNE

# Load the data
file_path = '/Users/nandini.gadhia/Documents/projects/gp_omics/data_rvc/OTU_table.csv'
df = pd.read_csv(file_path)

# Assuming the first column is an identifier and rest are features
if df.shape[1] > 1:
    features = df.iloc[:, 1:]
else:
    raise ValueError("Dataset does not have enough columns for dimensionality reduction.")

# Fit UMAP with a large number of components
initial_n_components = min(20, features.shape[1])
umap_reducer = umap.UMAP(n_components=initial_n_components, random_state=42)
umap_result = umap_reducer.fit_transform(features)

# Determine the number of components that preserve 90% variance
cumulative_variance = np.cumsum(np.var(umap_result, axis=0)) / np.sum(np.var(umap_result, axis=0))
n_components_90 = np.argmax(cumulative_variance >= 0.95) + 1

# Re-run UMAP with the optimal number of components
umap_reducer = umap.UMAP(n_components=80, random_state=42)
umap_result = umap_reducer.fit_transform(features)

# Create a DataFrame for the reduced data
umap_columns = [f'UMAP_{i+1}' for i in range(n_components_90)]
umap_df = pd.DataFrame(umap_result)


# Save the reduced data
umap_output_path = '/Users/nandini.gadhia/Documents/projects/gp_omics/data_rvc/OTU_table_umap.csv'

umap_df.to_csv(umap_output_path, index=False)

print(f"UMAP reduced data with {n_components_90} components saved to {umap_output_path}")


ImportError: Numba needs NumPy 2.1 or less. Got NumPy 2.2.

In [9]:
import os
import traceback
import numpy as np
import pandas as pd
from scipy.stats.mstats import gmean
import torch
import gpytorch
import matplotlib.pyplot as plt

def process_data(
    otu_file_path: str,
    metadata_file_path: str,
    sample_id: str = 'Sample',
    condition_id: str = 'Study.Group'
):
    """
    Processes OTU table and metadata, and applies a CLR transformation.
    """
    try:
        otu_table_raw = pd.read_csv(otu_file_path, index_col=0, sep='\t' if otu_file_path.endswith('.tsv') else ',')
        metadata = pd.read_csv(metadata_file_path, sep='\t' if metadata_file_path.endswith('.tsv') else ',')

        if sample_id not in metadata.columns:
            raise ValueError(f"Sample ID '{sample_id}' not found in metadata.")
        if condition_id not in metadata.columns:
            raise ValueError(f"Condition ID '{condition_id}' not found in metadata.")

        metadata.set_index(sample_id, inplace=True)
        common_samples = otu_table_raw.index.intersection(metadata.index)
        otu_table_aligned = otu_table_raw.loc[common_samples]
        metadata = metadata.loc[common_samples]
        otu_table_aligned = otu_table_aligned.reindex(metadata.index)

        # Add a pseudo-count to handle zeros before CLR transformation
        otu_table_no_zeros = otu_table_aligned + 1
        # Calculate geometric mean for each sample (row-wise)
        geometric_means = gmean(otu_table_no_zeros, axis=1)
        # Apply the CLR transformation
        otu_table_clr = np.log(otu_table_no_zeros.div(geometric_means, axis=0))

        feature_table = otu_table_clr
        feature_table = feature_table.loc[:, (feature_table != 0).any(axis=0)]

        metadata['condition'] = metadata.index.str[0] + '-' + metadata[condition_id].astype(str)

        return feature_table, metadata

    except FileNotFoundError:
        print(f"Error: One of the files was not found. Please check paths.")
        print(f"OTU Path: {otu_file_path}")
        print(f"Metadata Path: {metadata_file_path}")
        return None, None
    except Exception as e:
        print(f"An error occurred during data processing: {e}")
        traceback.print_exc()
        return None, None


otu_file = "/Users/nandini.gadhia/Documents/projects/ot_omics/notebooks/mock_counts.csv"
meta_file = "/Users/nandini.gadhia/Documents/projects/ot_omics/notebooks/mock_metadata.csv"

feature_table, metadata = process_data(
    otu_file, meta_file, sample_id='SampleID', condition_id='Group'
)
labels_binary = metadata['condition'].apply(lambda val: 0 if str(val).endswith('Control') else 1).values
features = feature_table.values

print("Data Loaded and Processed Successfully.")
print(f"Feature matrix shape (features): {features.shape}")
print(f"Labels array shape (labels_binary): {labels_binary.shape}")

print("\nData Loaded and Processed Successfully.")
print(f"Feature matrix shape (features): {features.shape}")
print(f"Labels array shape (labels_binary): {labels_binary.shape}")


Y = torch.tensor(features, dtype=torch.float32)
Y = (Y - Y.mean(dim=0)) / Y.std(dim=0)

n_points = Y.shape[0]
data_dim = Y.shape[1]
latent_dim = 80

# GPLVM model definition
class GPLVM(gpytorch.models.ExactGP):
    def __init__(self, n_points, latent_dim):
        # A placeholder for targets, which will be updated in the training loop
        train_x_placeholder = torch.randn(n_points, latent_dim)
        train_y_placeholder = torch.zeros(n_points)
        likelihood = gpytorch.likelihoods.GaussianLikelihood()
        
        super().__init__(train_x_placeholder, train_y_placeholder, likelihood)
        
        self.X = torch.nn.Parameter(torch.randn(n_points, latent_dim))

        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(ard_num_dims=latent_dim)
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

model = GPLVM(n_points, latent_dim)
targets = [Y[:, i] for i in range(data_dim)]


optimizer = torch.optim.Adam(model.parameters(), lr=0.2)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(model.likelihood, model)


student_t_prior = torch.distributions.StudentT(df=3.0, loc=0.0, scale=1.0)
laplace_prior = torch.distributions.Laplace(loc=0.0, scale=1.0)

n_iterations = 200
losses = []
for i in range(n_iterations):
    optimizer.zero_grad()
    
    # Calculate the log-likelihood part of the loss
    # This is the sum of MLLs over all output dimensions
    log_likelihood_loss = 0
    for j in range(data_dim):
        model.set_train_data(inputs=model.X, targets=targets[j], strict=False)
        output = model(model.X)
        log_likelihood_loss -= mll(output, targets[j])

    #log_prior_loss = -student_t_prior.log_prob(model.X).sum()
    log_prior_loss = -laplace_prior.log_prob(model.X).sum()


    # The total loss is the MAP objective: -log p(Y|X) - log p(X)
    #total_loss = log_likelihood_loss + log_prior_loss #use this one for laplace or student's t prior
    total_loss = log_likelihood_loss #use this one for Gaussian prior

    total_loss.backward()
    optimizer.step()
    if (i + 1) % 10 == 0:
        print(f"Iter {i+1}/{n_iterations} - Loss: {total_loss.item():.3f}")
        losses.append(total_loss.item())

print("Training finished.")

# Plotting the training loss curve
plt.figure(figsize=(8, 5))
plt.plot(losses)
plt.title("Training Loss Curve (MAP Objective)")
plt.xlabel("Iteration (x10)")
plt.ylabel("Loss (-Log Posterior)")
plt.grid(True)
plt.show()

model.eval()
with torch.no_grad():
    X_latent = model.X.detach().numpy()

plt.figure(figsize=(10, 8))
controls = X_latent[labels_binary == 0]
cases = X_latent[labels_binary == 1]

plt.scatter(controls[:, 0], controls[:, 1], c='blue', label='Control (0)', alpha=0.7)
plt.scatter(cases[:, 0], cases[:, 1], c='red', label='Case (1)', alpha=0.7)

plt.title("GPLVM: Learned 2D Latent Space with Student's T Prior", fontsize=16)
plt.xlabel("Latent Dimension 1", fontsize=12)
plt.ylabel("Latent Dimension 2", fontsize=12)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()


Data Loaded and Processed Successfully.
Feature matrix shape (features): (200, 100)
Labels array shape (labels_binary): (200,)

Data Loaded and Processed Successfully.
Feature matrix shape (features): (200, 100)
Labels array shape (labels_binary): (200,)
Iter 10/200 - Loss: 141.674
Iter 20/200 - Loss: 141.770
Iter 30/200 - Loss: 141.695
Iter 40/200 - Loss: 141.643
Iter 50/200 - Loss: 141.651
Iter 60/200 - Loss: 141.643
Iter 70/200 - Loss: 141.644
Iter 80/200 - Loss: 141.643
Iter 90/200 - Loss: 141.643
Iter 100/200 - Loss: 141.643
Iter 110/200 - Loss: 141.643
Iter 120/200 - Loss: 141.643


KeyboardInterrupt: 

In [7]:
X_latent_df = pd.DataFrame(X_latent, columns=[f'Latent_{i+1}' for i in range(latent_dim)])
X_latent_df.to_csv('/Users/nandini.gadhia/Documents/projects/gp_omics/data_for_yue/OTU_table_mock_data_latent_Normal.csv', index=False)